# Fila de caixa

#### Este estudo tem como objetivo a construção de um modelo analítico que fará a previsão do percentual de atendimentos com espera acima de 15 minutos, para auxiliar na criação de estratégias para a redução desse tempo.

#### Nos dias atuais, o dinamismo é um dos fatores-chave na relação entre pessoas e organizações. Qualquer tipo de atraso operacional é visto como uma característica negativa e acarreta o descontentamento por parte dos clientes.

#### A espera em uma fila de banco pode ser crítica, uma vez que o tempo percebido pelo consumidor pode ser maior que o analisado pela instituição. E a demora no atendimento é, muitas vezes, o motivo da insatisfação da percepção geral do atendimento.

#### Neste contexto, temos a seguinte situação: em Datacity temos 25 agências bancárias que possuem serviço de caixa. Pensando no bem-estar dos nossos clientes, seria importante saber o percentual de atendimento que foi iniciado com tempo de espera acima de 15 minutos. O estudo visa trazer informações para cada hora, sabendo que o horário de funcionamento é das 10h às 14h. Essas informações irão colaborar com a construção da estratégia de atuação, que tem como objetivo reduzir o tempo de atendimento.

### Para realizar a análise, foi utilizada uma base pública de dados, em formato csv, contendo as seguintes variáveis:
* ID - Código identificador único para cada protocolo gerado;
* ID_UOR - Código da agência, indicado por uma sequência de 0 a 24;
* TS_INC_EPR - Timestamp do início da espera, ou seja, a data e hora de emissão do protocolo;
* TS_INC_CHMD - Timestamp da chamada do protocolo

### Os protocolos foram agrupados por faixa de horário, tendo como referência a variável TS_INC_EPR
* DT - Recebe a data do atendimento;
* HH_INC - Recebe o horário de TS_INC_EPR;
* HH_FIM - Recebe o horário de TS_INC_CHMD;
* Exemplo de cálculo PC_AC_PZ (percentual acima do prazo):
** Dentre os protocolos emitidos entre 10h e 10:59:59, em determinado dia para determinada UOR, 8 (oito) foram atendidos em até 15 minutos  e 2 (dois) foram atendidos com espera superior a 15 minutos. A proporção atendida acima do prazo é 20%.

In [1]:
# Importando as bibliotecas.
import pandas as pd
import datetime
import numpy as np

In [2]:
# Lendo o dataframe.
dados = pd.read_csv('Dataset_publico.csv', sep=';')
dados.head()

,ID,ID_UOR,TS_INC_EPR,TS_INC_CHMD
0,0,0,2020-09-01 10:01:24,2020-09-01 10:02:13
1,1,0,2020-09-01 10:04:21,2020-09-01 10:22:01
2,2,0,2020-09-01 10:16:14,2020-09-01 10:25:06
3,3,0,2020-09-01 10:28:38,2020-09-01 10:28:40
4,4,0,2020-09-01 10:37:38,2020-09-01 10:37:41


In [3]:
dados["DT"] = dados["TS_INC_EPR"].str[:10]
dados["DT"] = pd.to_datetime(dados["DT"])

In [4]:
dados.head()

,ID,ID_UOR,TS_INC_EPR,TS_INC_CHMD,DT
0,0,0,2020-09-01 10:01:24,2020-09-01 10:02:13,2020-09-01
1,1,0,2020-09-01 10:04:21,2020-09-01 10:22:01,2020-09-01
2,2,0,2020-09-01 10:16:14,2020-09-01 10:25:06,2020-09-01
3,3,0,2020-09-01 10:28:38,2020-09-01 10:28:40,2020-09-01
4,4,0,2020-09-01 10:37:38,2020-09-01 10:37:41,2020-09-01


In [5]:
# Informações sobre o dataframe.

In [6]:
dados.shape

(335612, 5)

In [7]:
dados.describe()

,ID,ID_UOR
count,335612.000000,335612.000000
mean,187666.925459,10.849687
std,109062.064477,7.299419
min,0.000000,0.000000
25%,93401.750000,5.000000
50%,186928.500000,9.000000
75%,282781.250000,18.000000
max,376681.000000,24.000000


In [8]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335612 entries, 0 to 335611
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   ID           335612 non-null  int64         
 1   ID_UOR       335612 non-null  int64         
 2   TS_INC_EPR   335612 non-null  object        
 3   TS_INC_CHMD  335428 non-null  object        
 4   DT           335612 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 12.8+ MB


In [9]:
# Eliminando valores NaN.
dados = dados.dropna(axis=0, how='any')

In [10]:
# Conferindo alterações
dados.shape

(335428, 5)

In [11]:
# Convertendo as colunas "TS_INC_EPR" e "TS_INC_CHMD" para tipo datetime.

In [12]:
dados["TS_INC_EPR"] = pd.to_datetime(dados.TS_INC_EPR)

In [13]:
dados["TS_INC_CHMD"] = pd.to_datetime(dados.TS_INC_CHMD)

In [14]:
#Criando coluna "TS_TTL_EPR" que vai receber a diferença entre os valores das colunas "TS_INC_CHMD" 
# e "TS_INC_EPR", respectivamente.
dados["TS_TTL_EPR"] = dados["TS_INC_CHMD"] - dados["TS_INC_EPR"]

In [15]:
# Visualisando a nova configuração.
dados.head()

,ID,ID_UOR,TS_INC_EPR,TS_INC_CHMD,DT,TS_TTL_EPR
0,0,0,2020-09-01 10:01:24,2020-09-01 10:02:13,2020-09-01,0 days 00:00:49
1,1,0,2020-09-01 10:04:21,2020-09-01 10:22:01,2020-09-01,0 days 00:17:40
2,2,0,2020-09-01 10:16:14,2020-09-01 10:25:06,2020-09-01,0 days 00:08:52
3,3,0,2020-09-01 10:28:38,2020-09-01 10:28:40,2020-09-01,0 days 00:00:02
4,4,0,2020-09-01 10:37:38,2020-09-01 10:37:41,2020-09-01,0 days 00:00:03


In [16]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 335428 entries, 0 to 335611
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype          
---  ------       --------------   -----          
 0   ID           335428 non-null  int64          
 1   ID_UOR       335428 non-null  int64          
 2   TS_INC_EPR   335428 non-null  datetime64[ns] 
 3   TS_INC_CHMD  335428 non-null  datetime64[ns] 
 4   DT           335428 non-null  datetime64[ns] 
 5   TS_TTL_EPR   335428 non-null  timedelta64[ns]
dtypes: datetime64[ns](3), int64(2), timedelta64[ns](1)
memory usage: 17.9 MB


In [ ]:
#Criando as variáveis "Acima do prazo" e "Dentro do prazo"

In [18]:
def tempo(element): 
    if element.total_seconds() > (15 * 60):
        return 'Acima do prazo'
    else:
        return 'Dentro do prazo'

In [19]:
# Criando coluna "ESP" para receber os valores "Dentro do prazo" e "Acima do prazo"
dados['ESP'] = dados.TS_TTL_EPR.map(tempo)

In [20]:
# Conferindo inclusão da coluna "ESP" no dataframe dados
dados.head()

,ID,ID_UOR,TS_INC_EPR,TS_INC_CHMD,DT,TS_TTL_EPR,ESP
0,0,0,2020-09-01 10:01:24,2020-09-01 10:02:13,2020-09-01,0 days 00:00:49,Dentro do prazo
1,1,0,2020-09-01 10:04:21,2020-09-01 10:22:01,2020-09-01,0 days 00:17:40,Acima do prazo
2,2,0,2020-09-01 10:16:14,2020-09-01 10:25:06,2020-09-01,0 days 00:08:52,Dentro do prazo
3,3,0,2020-09-01 10:28:38,2020-09-01 10:28:40,2020-09-01,0 days 00:00:02,Dentro do prazo
4,4,0,2020-09-01 10:37:38,2020-09-01 10:37:41,2020-09-01,0 days 00:00:03,Dentro do prazo


In [ ]:
# Incluindo as colunas com os intervalos iniciais e finais dos horários

In [21]:
dados['HH_INC'] = dados['TS_INC_EPR'].apply(lambda x: str(x)[11:13]+':00:00')
dados.head()

,ID,ID_UOR,TS_INC_EPR,TS_INC_CHMD,DT,TS_TTL_EPR,ESP,HH_INC
0,0,0,2020-09-01 10:01:24,2020-09-01 10:02:13,2020-09-01,0 days 00:00:49,Dentro do prazo,10:00:00
1,1,0,2020-09-01 10:04:21,2020-09-01 10:22:01,2020-09-01,0 days 00:17:40,Acima do prazo,10:00:00
2,2,0,2020-09-01 10:16:14,2020-09-01 10:25:06,2020-09-01,0 days 00:08:52,Dentro do prazo,10:00:00
3,3,0,2020-09-01 10:28:38,2020-09-01 10:28:40,2020-09-01,0 days 00:00:02,Dentro do prazo,10:00:00
4,4,0,2020-09-01 10:37:38,2020-09-01 10:37:41,2020-09-01,0 days 00:00:03,Dentro do prazo,10:00:00


In [22]:
dados['HH_FIM'] = dados['TS_INC_EPR'].apply(lambda x: str(x)[11:13]+':59:59')
dados.head(50)

,ID,ID_UOR,TS_INC_EPR,TS_INC_CHMD,DT,TS_TTL_EPR,ESP,HH_INC,HH_FIM
0,0,0,2020-09-01 10:01:24,2020-09-01 10:02:13,2020-09-01,0 days 00:00:49,Dentro do prazo,10:00:00,10:59:59
1,1,0,2020-09-01 10:04:21,2020-09-01 10:22:01,2020-09-01,0 days 00:17:40,Acima do prazo,10:00:00,10:59:59
2,2,0,2020-09-01 10:16:14,2020-09-01 10:25:06,2020-09-01,0 days 00:08:52,Dentro do prazo,10:00:00,10:59:59
3,3,0,2020-09-01 10:28:38,2020-09-01 10:28:40,2020-09-01,0 days 00:00:02,Dentro do prazo,10:00:00,10:59:59
4,4,0,2020-09-01 10:37:38,2020-09-01 10:37:41,2020-09-01,0 days 00:00:03,Dentro do prazo,10:00:00,10:59:59
5,5,0,2020-09-01 10:39:35,2020-09-01 10:39:38,2020-09-01,0 days 00:00:03,Dentro do prazo,10:00:00,10:59:59
6,6,0,2020-09-01 10:41:34,2020-09-01 10:41:36,2020-09-01,0 days 00:00:02,Dentro do prazo,10:00:00,10:59:59
7,7,0,2020-09-01 10:44:22,2020-09-01 10:44:24,2020-09-01,0 days 00:00:02,Dentro do prazo,10:00:00,10:59:59
8,8,0,2020-09-01 10:48:41,2020-09-01 10:48:45,2020-09-01,0 days 00:00:04,Dentro do prazo,10:00:00,10:59:59
9,9,0,2020-09-01 10:49:57,2020-09-01 10:50:00,2020-09-01,0 days 00:00:03,Dentro do prazo,10:00:00,10:59:59


In [23]:
# Salvando as alterações em um novo dataset
dados.to_csv('Data Challengee.csv', index=False)

In [24]:
# Agrupando as senhas por hora
senhas_per_hour = dados.groupby(['ID_UOR','HH_INC']).count()['ID'].to_frame()
senhas_per_hour

ID
ID_UOR HH_INC        
0      09:00:00    17
       10:00:00  4461
       11:00:00  3650
       12:00:00  3589
       13:00:00  3553
...               ...
24     11:00:00  1007
       12:00:00   626
       13:00:00  1079
       14:00:00   428
       15:00:00    37

[192 rows x 1 columns]

In [25]:
senhas_per_hour.to_csv()

'ID_UOR,HH_INC,ID\n0,09:00:00,17\n0,10:00:00,4461\n0,11:00:00,3650\n0,12:00:00,3589\n0,13:00:00,3553\n0,14:00:00,2215\n0,15:00:00,224\n0,16:00:00,1\n1,08:00:00,8\n1,09:00:00,1877\n1,10:00:00,5956\n1,11:00:00,4951\n1,12:00:00,4354\n1,13:00:00,5498\n1,14:00:00,1626\n1,15:00:00,42\n2,08:00:00,1\n2,09:00:00,192\n2,10:00:00,3590\n2,11:00:00,2732\n2,12:00:00,2551\n2,13:00:00,3028\n2,14:00:00,1856\n2,15:00:00,207\n2,16:00:00,1\n4,08:00:00,1\n4,09:00:00,569\n4,10:00:00,4792\n4,11:00:00,3591\n4,12:00:00,3131\n4,13:00:00,3524\n4,14:00:00,1838\n4,15:00:00,206\n5,09:00:00,30\n5,10:00:00,1516\n5,11:00:00,4543\n5,12:00:00,3672\n5,13:00:00,3626\n5,14:00:00,2789\n5,15:00:00,2652\n5,16:00:00,457\n5,17:00:00,2\n6,09:00:00,16\n6,10:00:00,1800\n6,11:00:00,3663\n6,12:00:00,3236\n6,13:00:00,3124\n6,14:00:00,1939\n6,15:00:00,1774\n6,16:00:00,79\n7,09:00:00,1\n7,10:00:00,127\n7,11:00:00,159\n7,12:00:00,173\n7,13:00:00,137\n7,14:00:00,39\n7,15:00:00,1\n8,09:00:00,72\n8,10:00:00,2782\n8,11:00:00,11854\n8,12:00:

In [ ]:
# Separando os horários de retirada de senha e atendimento da data

In [39]:
dados['TS_INC_EPR'] = pd.to_datetime(dados['TS_INC_EPR'],format= '%H:%M:%S' ).dt.time
dados.head()

,ID,ID_UOR,TS_INC_EPR,TS_INC_CHMD,DT,TS_TTL_EPR,ESP,HH_INC,HH_FIM
0,0,0,10:01:24,2020-09-01 10:02:13,2020-09-01,0 days 00:00:49,Dentro do prazo,10:00:00,20:59:59
1,1,0,10:04:21,2020-09-01 10:22:01,2020-09-01,0 days 00:17:40,Acima do prazo,10:00:00,20:59:59
2,2,0,10:16:14,2020-09-01 10:25:06,2020-09-01,0 days 00:08:52,Dentro do prazo,10:00:00,20:59:59
3,3,0,10:28:38,2020-09-01 10:28:40,2020-09-01,0 days 00:00:02,Dentro do prazo,10:00:00,20:59:59
4,4,0,10:37:38,2020-09-01 10:37:41,2020-09-01,0 days 00:00:03,Dentro do prazo,10:00:00,20:59:59


In [40]:
dados['TS_INC_CHMD'] = pd.to_datetime(dados['TS_INC_CHMD'],format= '%H:%M:%S' ).dt.time
dados.head()

,ID,ID_UOR,TS_INC_EPR,TS_INC_CHMD,DT,TS_TTL_EPR,ESP,HH_INC,HH_FIM
0,0,0,10:01:24,10:02:13,2020-09-01,0 days 00:00:49,Dentro do prazo,10:00:00,20:59:59
1,1,0,10:04:21,10:22:01,2020-09-01,0 days 00:17:40,Acima do prazo,10:00:00,20:59:59
2,2,0,10:16:14,10:25:06,2020-09-01,0 days 00:08:52,Dentro do prazo,10:00:00,20:59:59
3,3,0,10:28:38,10:28:40,2020-09-01,0 days 00:00:02,Dentro do prazo,10:00:00,20:59:59
4,4,0,10:37:38,10:37:41,2020-09-01,0 days 00:00:03,Dentro do prazo,10:00:00,20:59:59


In [43]:
# Imprimindo a frequência dos atendimentos realizados dentro do prazo e acima do prazo
freq = dados['ESP'].value_counts() 
print(freq)

Dentro do prazo    304987
Acima do prazo      30441
Name: ESP, dtype: int64


In [44]:
percentual = dados['ESP'].value_counts(normalize=True)

In [45]:
percentual

Dentro do prazo    0.909247
Acima do prazo     0.090753
Name: ESP, dtype: float64

In [46]:
dados.to_csv('./data/dados.csv',sep=',',index=False)